<a href="https://colab.research.google.com/github/ssnirgudkar/Datasetpaper-final/blob/main/DownloadDataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import zipfile
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Remove files from this folder in google drive so there is no duplication
#!rm -rf /content/drive/MyDrive/DatasetMetricReport/revised_json/*.json 

In [ ]:
## This script is used to download the segmented images from segments.ai
## Once a release is created, all the mask files can be downloaded with this script
## The download also consists of original images.
## Please make sure that you install latest segments.ai because they keep changing API arguments
## From command shell (administrative) pip install segments-ai --upgrade
## The key given to SegmentsClient is a unique key generated for me.
## E.g. On mlpc, use python3 DownloadSegmentedImages.py ( DO NOT USE python, USE python3)
!pip install segments-ai --upgrade

from segments import SegmentsClient, SegmentsDataset
from segments.utils import export_dataset
import sys
import argparse
import json
import requests
import os



def downloadDataset(account_name, dataset_path, version, export_format, userlabel, client):
    full_dataset_path = account_name + '/' + dataset_path
    release_file = client.get_release(full_dataset_path, version) 
    print(release_file)
    url = (requests.get(release_file['attributes']['url']))
    print(url)
    content = json.loads(url.text)

    json_file_name = dataset_path + '-' + version + '.json'
    with open(json_file_name, "w") as json_file:
        json.dump(content, json_file)
    #print("JSON file contents={0}".format(content))

    release_dataset = SegmentsDataset(release_file, labelset=userlabel, filter_by=['labeled', 'reviewed'])
    # Export to COCO format
    json_file = ''
    downloadDir = ''
    if ('coco' in export_format):
        (json_file, downloadDir) = export_dataset(release_dataset, export_format=export_format)
    else:
        downloadDir = export_dataset(release_dataset, export_format=export_format)
    print("Images downloaded to dir = {0}".format(downloadDir))

    # Rename each file from XX.YY.png to aXX_YY.png
    for eachFile in os.listdir(downloadDir):
        newName = eachFile.replace('.', '_', 1) # replace only first occurrence of first '.'
        newName = "a" + newName
        os.rename(os.path.join(downloadDir, eachFile), os.path.join(downloadDir, newName))

    (dir, leafDir) = os.path.split(downloadDir)
    newLeafDir = leafDir + '_' + export_format
    newDirPath = os.path.join(dir, newLeafDir)
    os.rename(downloadDir, newDirPath)
    print("Directory renamed to = {0}".format(newDirPath))
    print("Dataset downloaded successfully.")
    return json_file_name
        

def main(args):
    account_name = None
    if args['account_name'] is None:
        print('Specify account name')
        sys.exit(1)
    else:
        print("account name = {0}".format(args['account_name']))
        account_name = args['account_name']

    dataset_path_list = None
    if args['dataset_path'] is None:
        print('Specify dataset path')
        sys.exit(1)
    else:
        print("dataset path = {0}".format(args['dataset_path']))
        dataset_path_list = args['dataset_path']

    version = None
    if args['version'] is None:
        print('Specify version')
        sys.exit(1)
    else:
        print("version = {0}".format(args['version']))
        version = args['version']

    userlabel = None
    if args['userlabel'] is None:
        print('Specify userlabel (ground-truth) or segmentation')
        sys.exit(1)
    else:
        print("user label = {0}".format(args["userlabel"]))
        userlabel = args["userlabel"]


    # Initialize a SegmentsDataset from the release file
    if (account_name == "ssnirgudkar"):
        client = SegmentsClient('a89182567b17766b91773021b18d04574cd75109')
    if (account_name == "brunswick"):
        client = SegmentsClient('98e631fa8e12a9a4d998b999d6f8e25aa2feb358')

    json_file_names = []
    if (dataset_path_list == "all"):
        datasets = client.get_datasets()
        for dataset in datasets:
            dataset_path = dataset["name"]
            if (dataset_path == "playground"):
                continue
            print("dataset:{0},version:{1}".format(dataset_path,version))
            json_file_name = downloadDataset(account_name, dataset_path, version, args['export_format'], userlabel, client)
            json_file_names.append(json_file_name)
    else:
        dataset_paths = dataset_path_list.split(',')
        for dataset_path in dataset_paths:
            json_file_name = downloadDataset(account_name, dataset_path, version, args['export_format'], userlabel, client)
            json_file_names.append(json_file_name)

    return json_file_names





    

#release_file = 'IR-2019-10-15-rex4usv-13-38-24.json'
#dataset = SegmentsDataset(release_file, labelset='ground-truth', filter_by=['labeled'])

    

# To visualize the masks, following code can be used.
# import matplotlib.pyplot as plt
# from segments.utils import get_semantic_bitmap
'''
for sample in release_dataset1:
    # Print the sample name and list of labeled objects
    print(sample['name'])
    print(sample['annotations'])
    
    # Show the image
    plt.imshow(sample['image'])
    plt.show()
    
    # Show the instance segmentation label
    plt.imshow(sample['segmentation_bitmap'])
    plt.show()
    
    # Show the semantic segmentation label
    semantic_bitmap = get_semantic_bitmap(sample['segmentation_bitmap'], sample['annotations'])
    plt.imshow(semantic_bitmap)
    plt.show()
'''    


In [5]:
# Pull Non Pilot IR dataset from segments-ai. Pulls both images and json files 
args = {}
args["account_name"] = "ssnirgudkar"
args["dataset_path"] = "IR-2019-12-11-11-09-59,IR-2019-10-22-11-56-01,IR-2020-10-20-16-32-52-1,IR-2019-10-28-philios-16-25-00,IR-2020-10-22-17-33-25-0,IR-2020-10-20-15-52-18"
args["version"] = "v2.0"
args["export_format"] = "coco-panoptic"
args["userlabel"] = "ground-truth"

main(args)

account name = ssnirgudkar
dataset path = IR-2019-12-11-11-09-59,IR-2019-10-22-11-56-01,IR-2020-10-20-16-32-52-1,IR-2019-10-28-philios-16-25-00,IR-2020-10-22-17-33-25-0,IR-2020-10-20-15-52-18
version = v2.0
user label = ground-truth
Initialized successfully.
{'uuid': 'e0570cd3-0090-4c50-903c-537acfe2323c', 'name': 'v2.0', 'description': 'Final release for December', 'release_type': 'JSON', 'attributes': {'url': 'https://segmentsai-prod.s3.eu-west-2.amazonaws.com/releases/e0570cd3-0090-4c50-903c-537acfe2323c.json'}, 'status': 'SUCCEEDED', 'status_info': '', 'created_at': '2021-12-06T13:07:34.475403Z', 'samples_count': 9}
<Response [200]>
Initializing dataset...
Preloading all samples. This may take a while...


100%|██████████| 9/9 [00:02<00:00,  3.24it/s]


Initialized dataset with 9 images.
Exporting dataset. This may take a while...


100%|██████████| 9/9 [00:00<00:00, 21.41it/s]


Exported to ./export_coco-panoptic_ssnirgudkar_IR-2019-12-11-11-09-59_v2.0.json. Images and labels in segments/ssnirgudkar_IR-2019-12-11-11-09-59/v2.0
Images downloaded to dir = segments/ssnirgudkar_IR-2019-12-11-11-09-59/v2.0
Directory renamed to = segments/ssnirgudkar_IR-2019-12-11-11-09-59/v2.0_coco-panoptic
Dataset downloaded successfully.
{'uuid': '972cd482-144f-4d6c-86ad-bea85c835d84', 'name': 'v2.0', 'description': 'Final release for December', 'release_type': 'JSON', 'attributes': {'url': 'https://segmentsai-prod.s3.eu-west-2.amazonaws.com/releases/972cd482-144f-4d6c-86ad-bea85c835d84.json'}, 'status': 'SUCCEEDED', 'status_info': '', 'created_at': '2021-12-06T13:02:44.720497Z', 'samples_count': 16}
<Response [200]>
Initializing dataset...
Preloading all samples. This may take a while...


...

100%|██████████| 41/41 [00:06<00:00,  6.82it/s]


Initialized dataset with 41 images.
Exporting dataset. This may take a while...


100%|██████████| 41/41 [00:02<00:00, 18.31it/s]

Exported to ./export_coco-panoptic_ssnirgudkar_IR-2020-10-20-15-52-18_v2.0.json. Images and labels in segments/ssnirgudkar_IR-2020-10-20-15-52-18/v2.0
Images downloaded to dir = segments/ssnirgudkar_IR-2020-10-20-15-52-18/v2.0
Directory renamed to = segments/ssnirgudkar_IR-2020-10-20-15-52-18/v2.0_coco-panoptic
Dataset downloaded successfully.


['IR-2019-12-11-11-09-59-v2.0.json',
 'IR-2019-10-22-11-56-01-v2.0.json',
 'IR-2020-10-20-16-32-52-1-v2.0.json',
 'IR-2019-10-28-philios-16-25-00-v2.0.json',
 'IR-2020-10-22-17-33-25-0-v2.0.json',
 'IR-2020-10-20-15-52-18-v2.0.json']

In [6]:
# Pull Pilot IR dataset from segments-ai. Pulls both images and json files 
args = {}
args["account_name"] = "ssnirgudkar"
args["dataset_path"] = "PilotIR"
args["version"] = "v2.0"
args["export_format"] = "coco-panoptic"
args["userlabel"] = "segmentation"

main(args)

account name = ssnirgudkar
dataset path = PilotIR
version = v2.0
user label = segmentation
Initialized successfully.
{'uuid': 'd0237292-6316-4eef-80d0-c95bf179994c', 'name': 'v2.0', 'description': 'Final release for December', 'release_type': 'JSON', 'attributes': {'url': 'https://segmentsai-prod.s3.eu-west-2.amazonaws.com/releases/d0237292-6316-4eef-80d0-c95bf179994c.json'}, 'status': 'SUCCEEDED', 'status_info': '', 'created_at': '2022-01-01T21:51:43.646631Z', 'samples_count': 103}
<Response [200]>
Initializing dataset...
Preloading all samples. This may take a while...


100%|██████████| 79/79 [00:08<00:00,  9.19it/s]


Initialized dataset with 79 images.
Exporting dataset. This may take a while...


100%|██████████| 79/79 [00:05<00:00, 15.09it/s]


Exported to ./export_coco-panoptic_ssnirgudkar_PilotIR_v2.0.json. Images and labels in segments/ssnirgudkar_PilotIR/v2.0
Images downloaded to dir = segments/ssnirgudkar_PilotIR/v2.0
Directory renamed to = segments/ssnirgudkar_PilotIR/v2.0_coco-panoptic
Dataset downloaded successfully.


['PilotIR-v2.0.json']

In [7]:
# Pull brunswick from segments-ai. Pulls both images and json files. There are 417 combined from ss nirgudkar 
args = {}
args["account_name"] = "brunswick"
args["dataset_path"] = "all"
args["version"] = "v2.0"
args["export_format"] = "coco-panoptic"
args["userlabel"] = "ground-truth"

main(args)

account name = brunswick
dataset path = all
version = v2.0
user label = ground-truth
Initialized successfully.
dataset:IR-2019-10-29-philios-16-00-19,version:v2.0
{'uuid': '130c6ba4-7f75-4a76-a5ca-0964e7f742fe', 'name': 'v2.0', 'description': 'Final release for December', 'release_type': 'JSON', 'attributes': {'url': 'https://segmentsai-prod.s3.eu-west-2.amazonaws.com/releases/130c6ba4-7f75-4a76-a5ca-0964e7f742fe.json'}, 'status': 'SUCCEEDED', 'status_info': '', 'created_at': '2021-11-26T22:19:37.415705Z', 'samples_count': 69}
<Response [200]>
Initializing dataset...
Preloading all samples. This may take a while...


100%|██████████| 69/69 [00:08<00:00,  8.10it/s]


Initialized dataset with 69 images.
Exporting dataset. This may take a while...


100%|██████████| 69/69 [00:03<00:00, 17.66it/s]


Exported to ./export_coco-panoptic_brunswick_IR-2019-10-29-philios-16-00-19_v2.0.json. Images and labels in segments/brunswick_IR-2019-10-29-philios-16-00-19/v2.0
Images downloaded to dir = segments/brunswick_IR-2019-10-29-philios-16-00-19/v2.0
Directory renamed to = segments/brunswick_IR-2019-10-29-philios-16-00-19/v2.0_coco-panoptic
Dataset downloaded successfully.
dataset:IR-2019-10-08-rex4usv-13-29-51,version:v2.0
{'uuid': '86c9b298-334f-41d1-9105-8f61f6a5bba8', 'name': 'v2.0', 'description': 'Final release for December', 'release_type': 'JSON', 'attributes': {'url': 'https://segmentsai-prod.s3.eu-west-2.amazonaws.com/releases/86c9b298-334f-41d1-9105-8f61f6a5bba8.json'}, 'status': 'SUCCEEDED', 'status_info': '', 'created_at': '2021-11-28T00:21:03.251921Z', 'samples_count': 119}
<Response [200]>
Initializing dataset...
Preloading all samples. This may take a while...


...

100%|██████████| 33/33 [00:04<00:00,  6.88it/s]


Initialized dataset with 33 images.
Exporting dataset. This may take a while...


100%|██████████| 33/33 [00:01<00:00, 19.68it/s]

Exported to ./export_coco-panoptic_brunswick_IR-2021-10-28-16-19-00_0_v2.0.json. Images and labels in segments/brunswick_IR-2021-10-28-16-19-00_0/v2.0
Images downloaded to dir = segments/brunswick_IR-2021-10-28-16-19-00_0/v2.0
Directory renamed to = segments/brunswick_IR-2021-10-28-16-19-00_0/v2.0_coco-panoptic
Dataset downloaded successfully.


['IR-2019-10-29-philios-16-00-19-v2.0.json',
 'IR-2019-10-08-rex4usv-13-29-51-v2.0.json',
 'IR-2019-10-29-16-08-13-v2.0.json',
 'IR-2019-10-29-log-philios-15-55-36-v2.0.json',
 'IR-2019-10-08-14-49-48-v2.0.json',
 'IR-2019-10-08-15-05-15-v2.0.json',
 'IR-2019-11-05-philios-13-35-55-v2.0.json',
 'IR-2020-10-22-18-33-26-2-v2.0.json',
 'IR-2019-10-22-11-21-56-v2.0.json',
 'IR-2019-10-25-14-21-23-v2.0.json',
 'IR-2019-11-04-14-32-22-v2.0.json',
 'IR-2019-10-22-13-34-07-v2.0.json',
 'IR-2020-10-22-16-46-33-v2.0.json',
 'IR-2019-12-11-10-58-24-v2.0.json',
 'IR-2020-10-20-16-02-52-v2.0.json',
 'IR-2019-10-15-15-28-30-v2.0.json',
 'IR-2019-10-15-15-26-47-v2.0.json',
 'IR-2019-10-15-15-22-06-v2.0.json',
 'IR-2020-10-22-18-03-25-v2.0.json',
 'IR-2019-10-22-13-02-56-v2.0.json',
 'IR-2020-11-06-v2.0.json',
 'IR-2019-10-22-13-19-39-v2.0.json',
 'IR-2019-10-22-12-24-37-v2.0.json',
 'IR-2019-10-22-12-49-38-v2.0.json',
 'IR-2019-11-04-14-49-42-v2.0.json',
 'IR-2019-12-11-11-07-30-v2.0.json',
 'IR-2019

In [8]:
import os 
APP_FOLDER = '/content/segments'
totalFiles = 0
totalDir = 0

for base, dirs, files in os.walk(APP_FOLDER):
    print('Searching in : ',base)
    for directories in dirs:
        totalDir += 1
    for Files in files:
        if "_coco-panoptic.png" in Files:
          totalFiles += 1
   

print('Total number of Images in both brunswick and ssnirgudkar are',totalFiles)
print('Total Number of directories',totalDir)
print('Total:',(totalDir + totalFiles))

Searching in :  /content/segments
Searching in :  /content/segments/brunswick_IR-2020-10-22-16-46-33
Searching in :  /content/segments/brunswick_IR-2020-10-22-16-46-33/v2.0_coco-panoptic
Searching in :  /content/segments/brunswick_IR-catch-all
Searching in :  /content/segments/brunswick_IR-catch-all/v2.0_coco-panoptic
Searching in :  /content/segments/brunswick_IR-2019-10-22-13-19-39
Searching in :  /content/segments/brunswick_IR-2019-10-22-13-19-39/v2.0_coco-panoptic
Searching in :  /content/segments/brunswick_IR-RT-2020-10-20-15-39-42_0
Searching in :  /content/segments/brunswick_IR-RT-2020-10-20-15-39-42_0/v2.0_coco-panoptic
Searching in :  /content/segments/brunswick_IR-2020-10-20-15-39-42-0
Searching in :  /content/segments/brunswick_IR-2020-10-20-15-39-42-0/v2.0_coco-panoptic
Searching in :  /content/segments/brunswick_IR-2019-10-15-rex4usv-15-02-31
Searching in :  /content/segments/brunswick_IR-2019-10-15-rex4usv-15-02-31/v2.0_coco-panoptic
Searching in :  /content/segments/brun

In [9]:
# copy the select_test_images and masks in test and testannot respectively 


!cp /content/export_coco-panoptic*.json /content/drive/MyDrive/DatasetMetricReport/revised_json 

